# 用Jupyter玩MicroPython小車

## ESP12的電機板

- 電機電源（VM）：4.5V～36V，可單獨供電
- 控制電源（VIN）：4.5V～9V（10VMAX），可單獨供電
- 但我們這邊把VM, VIN短路，比較省事
- 其中，D1，D3（A電機）；D2，D4（B電機）

![電機板](image/motor_board.png)

### 供電

- 以行動電源供電即可

### 馬達

- TT減速馬達(48:1)

### 超音波測距模組

- HC-SR04+ : 給MicroPython 記得使用這一款，可以吃3.3V、5V的電，因為MicroPython是3.3V的GPIO

----

## 測試馬達

In [2]:
%serialconnect to --port=com13 --baud=115200

Connecting to --port=com13 --baud=115200 
Ready.


In [1]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: Ready.


In [2]:
# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM


In [3]:
#Setup PINS
led = machine.Pin(2, machine.Pin.OUT)
led.value(1)

In [4]:
# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 0
REV = 1


In [5]:
def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)


### 測試前進、後退、左轉、右轉

In [19]:
# 前進、後退
car_fwd()
time.sleep(3)
car_rev()
time.sleep(3)
car_stop()

.

In [6]:
car_fwd()

In [7]:
car_rev()

In [8]:
car_stop()

In [30]:
car_right()

In [31]:
car_left()

In [32]:
#全部動作跑一次
car_fwd()
time.sleep(2)
car_right()
time.sleep(2)
car_left()
time.sleep(2)
car_rev()
time.sleep(2)
car_stop()

.

----

## 超音波 test

- HC-SR04


In [147]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: Ready.


In [33]:
# 定義
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM


In [36]:
# 340m/s --> 340000mm/s --> 1mm需要2.94 us
# 340m/s --> 34000cm/s --> 1cm要29.4us --> 去回，所以除以(29.4*2)=58.8，但手冊是寫58us為1cm
# 所以如果目標是50cm內的量測距離 --> 50*58=2900us為最長時間
# timeout回傳0，很怪，應該給個大數字比較好，比如999

import time
from machine import Pin

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

while True:
    distance=ping(trigPin=D7,echoPin=D8)
    print('%s cm' % distance)
    time.sleep(1)

6.87931 cm
10.2414 cm
7.43103 cm
6.65517 cm
10.2414 cm
.7.43103 cm
6.65517 cm
10.2414 cm
9.46552 cm
999 cm
15.1552 cm
.14.3965 cm
15.1724 cm
13.1724 cm
14.3793 cm
20.5345 cm
.15.1552 cm
12.4828 cm
15.1552 cm
12.4828 cm
15.1724 cm
12.5 cm
15.1552 cm
.12.4828 cm


*** Sending Ctrl-C

15.1552 cm


Traceback (most recent call last):
  File "<stdin>", line 48, in <module>
KeyboardInterrupt: 


----

### 燒入MicroPython板子

- MicroPython一上電會執行 main.py 的程式，因此我們使用magic command "sendtofile" 燒入板子裡


In [16]:
%websocketconnect --password 1234 ws://192.168.43.245:8266

** WebSocket connected **
Password: Ready.


- 燒錄避障小車程式

In [17]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(2, machine.Pin.OUT)
for i in range(5):
    led.value(not led.value())

# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 0
REV = 1

def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)


def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

# go~
car_fwd()

while(True):
    distance=ping(trigPin=D7,echoPin=D8)
    if distance>25:
        car_fwd()
    elif distance>15:
        car_stop()
        time.sleep(0.2)
        car_right()
        time.sleep(0.5)
    elif distance<=10:
        car_stop()
        time.sleep(0.2)
        car_rev()
        time.sleep(0.5)
    time.sleep(0.1)

Sent 115 lines (2443 bytes) to main.py.


### 遙控車模式

In [ ]:
# UDP 測試



In [ ]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS (blink Led --> firmware start)
led = machine.Pin(D4, machine.Pin.OUT)
for i in range(5):
    led.value(not led.value())
    time.sleep(0.2)

# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 0
REV = 1

def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)


def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

# go~
car_stop()

while(True):
    distance=ping(trigPin=D7,echoPin=D8)
    if distance>25:
        car_fwd()
    elif distance>15:
        car_stop()
        time.sleep(0.2)
        car_right()
        time.sleep(0.5)
    elif distance<=10:
        car_stop()
        time.sleep(0.2)
        car_rev()
        time.sleep(0.5)
    time.sleep(0.1)